# Figure S1 - effect of different spatial resolutions

In [ ]:
import neuron
import numpy as np
import LFPy
import MEAutility as mu
import matplotlib.pyplot as plt
import neuroplotlib as nplt
from pathlib import Path
import sys
import os
from pprint import pprint

In [ ]:
from axon_velocity import *
from axon_velocity.models import *
from axon_velocity.evaluation import *

%matplotlib widget

In [ ]:
save_figures = True
save_results = False

In [ ]:
fig_folder = Path('figures') / "figureSupp1"
fig_folder.mkdir(exist_ok=True, parents=True)

In [ ]:
try:
    import neuron
except:
    print('NEURON is not installed.')

mechanism_folder = Path('..') / 'simulations' / 'mechanisms'

if not neuron.load_mechanisms(str(mechanism_folder)):
    print('Compile mod files in the mechanisms/ folder: from the mechanisms/ folder, run nrnivmodl')

In [ ]:
params_dict = get_default_biophysics_params()
pprint(params_dict)

In [ ]:
planar = True
z_offset = 10 # distance between cell plane and mea plane
zspan = 0

In [ ]:
morphology_dir = Path('..') / 'simulations' / 'neuromorpho' / 'allen_cell_types'

morph_id = '561096006'
original_morphology_path = [m for m in morphology_dir.iterdir() if not 
                            m.name.startswith('.') and morph_id in str(m)][0]
if planar:
    morphology_path = planarize_swc(original_morphology_path, span_um=zspan)
else:
    morphology_path = original_morphology_path

In [ ]:
cell = LFPy.Cell(str(morphology_path), v_init=params_dict['v_init'], celsius=params_dict['celsius'],
                 Ra=params_dict['ra'], cm=params_dict['cm'], pt3d=True)

In [ ]:
# center in the xy plane
center_cell_xy(cell)

In [ ]:
# simple biophysiscs: dendrite - pas / soma/axon HH
# "complex" biophysics: dendrite - pas / soma - na + kv1 / axon - nax + kv1 
simple_biophysics = False

In [ ]:
if simple_biophysics:
    insert_simple_biophysics(cell)
else:
    insert_biophysics(cell, params_dict)

At this stage, one can also change the axial conductance (e.g. `sec.ra`), 
which likely affects the conduction velocity.

The `planar` variable decides wheter the z-axis is compressed (similar to a cell culture - `planar=True`) or the original morphology is used (`planar=False`).

In [ ]:
shift = z_offset
    
print(f"z-position of MEA: {shift}")

### Stimulating the cell

We can now add some stimulation. The stimulation can be a current clamp `iclamp` or synaptic inputs `syn`. The `stim_point` is where the cell will be stimulated (the closest cell segment to the `stim_point` is used).

In [ ]:
stim = 'syn' # or syn
# stimulate on the soma
stim_idx = cell.somaidx

syn_input_times = np.arange(2, 5)

syn_params = {'idx' : stim_idx,
              'e' : 0,                                # reversal potential
              'syntype' : 'ExpSyn',                   # synapse type
              'tau' : 2,                              # syn. time constant ms
              'weight' : 0.05,                         # syn. weight
              'record_current' : True                 # syn. current record
    }
clamp_params = {'idx' : stim_idx,
                'pptype' : 'IClamp',                   # IClamp point process
                'dur' : 300,                            # dur in ms
                'amp' : 2,                             # amp in nA
                'delay' : 5                            # delay in ms
    }

In [ ]:
if stim == 'syn':
    synapse = LFPy.Synapse(cell, **syn_params)
    synapse.set_spike_times(np.array(syn_input_times))
else:
    clamp = LFPy.StimIntElectrode(cell=cell, **clamp_params)

### Define extracellular electrodes

Let's now define the extracellular electrodes using the [MEAutility](https://meautility.readthedocs.io/en/latest/) package.

In [ ]:
pitches = [17.5, 35, 70, 140]
mea_size = 17.5*100

In [ ]:
elec_size = 8

In [ ]:
electrodes = []
probes = []
for p in pitches:
    mea_dim = int(mea_size / p)  # n rows x n cols
    mea_pitch = p  # rows and cols pitch

    mea_info = {'dim': mea_dim,
     'electrode_name': 'hd-mea',
     'pitch': mea_pitch,
     'shape': 'square',
     'size': elec_size, 
     'type': 'mea',
     'plane': 'xy'}

    hdmea = mu.return_mea(info=mea_info)
    
    # Move the MEA out of the neuron plane (yz)
    hdmea.move([0, 0, shift])
    
    probes.append(hdmea)
    # Instantiate LFPy electrode object
    electrode = LFPy.RecExtElectrode(cell, probe=hdmea, n=10)
    electrodes.append(electrode)

In [ ]:
figs_density = {}
for probe in probes:
    fig, ax = plt.subplots(figsize=(10, 10))
    pitch = probe.pitch[0]
    ax = mu.plot_probe(probe, type='planar', ax=ax)
    ax = nplt.plot_neuron(cell, plane='xy', color='k', ax=ax)
    ax.axis('off')
    figs_density[str(pitch)] = fig

In [ ]:
if save_figures:
    for pitch, fig in figs_density.items():
        fig.savefig(fig_folder / f"{pitch}_morphology.png", dpi=600)
        fig.savefig(fig_folder / f"{pitch}_morphology.pdf")

### Run the simulation

By passing the `electrode` argument `LFPy` also computes extracellular potentials. The `rec_vmem` argument allows to measure the membrane potenrtial at all segments.

In [ ]:
cell.simulate(probes=electrodes, rec_vmem=True)

In [ ]:
eaps = [electrode.data * 1000 for electrode in electrodes]  # mV --> uV

In [ ]:
# cutout single template
fs = 1 / cell.dt * 1000
ms_before = 2
ms_after = 10

min_chan, min_idx = np.unravel_index(np.argmin(eaps[0]), eaps[0].shape)

In [ ]:
# eaps_cut = [eap[:, min_idx - int(ms_before * fs): min_idx + int(ms_after * fs)] for eap in eaps]

In [ ]:
figs_amps = {}
for (probe, eap) in zip(probes, eaps):
    fig, ax = plt.subplots(figsize=(10, 10))
    pitch = probe.pitch[0]
    locations = probe.positions[:, :-1]
    ax2 = plot_amplitude_map(eap, locations, ax=ax, log=True, colorbar=False, 
                             cmap="PRGn", colorbar_orientation="horizontal", plot_image=False,
                             elec_size=2*pitch)
    ax.axis('off')
    ax.set_title(f"Pitch: {pitch}$\mu$m", fontsize=20)
    figs_amps[str(pitch)] = fig

In [ ]:
if save_figures:
    for pitch, fig in figs_amps.items():
        fig.savefig(fig_folder / f"{pitch}_amps.png", dpi=600)
        fig.savefig(fig_folder / f"{pitch}_amps.pdf")

## Run tracking algorithm

In [ ]:
params = get_default_graph_velocity_params()

# change params
params['detect_threshold'] = 0.01
params['kurt_threshold'] = 0.5
params['peak_std_threshold'] = None
params['init_delay'] = 0.1
params['upsample'] = 5
params['neighbor_radius'] = 50
params["r2_threshold"] = 0.9
params["max_distance_for_edge"] = 200
params["max_distance_to_init"] = 300
params["max_peak_latency_for_splitting"] = 0.5
params["min_selected_points"] = 5
params["remove_isolated"] = True

pprint(params)

In [ ]:
gtrs = []
for (eap, probe) in zip(eaps, probes):
    print(f"\n\n Pitch {probe.pitch[0]} \n\n")
    locations = probe.positions[:, :-1]
    params['neighbor_radius'] = 2 * probe.pitch[0]
    gtr = compute_graph_propagation_velocity(eap, locations, fs=fs, verbose=True, **params)
    gtrs.append(gtr)

In [ ]:
# branch_gt = extract_ground_truth_velocity(cell, min_length=50, min_segs=5)

In [ ]:
# evals = []
# for i, gtr in enumerate(gtrs):
#     ev = evaluate_tracking_accuracy(gtr.branches, branch_gt, cell, gtr.locations,
#                                     max_median_dist_for_match=50)
    
#     print(f"Num matches for pitch {probes[i].pitch[0]}: {len(ev)}")
#     evals.append(ev)

In [ ]:
cell_model = cell
cmap_branches = "tab20"
cmap_footprint = "Greys"
alpha_footprint = 0.5
alpha_marker = 0.7
legend_fs = 18

max_branches = np.max([len(gtr.branches) for gtr in gtrs])
cm = plt.get_cmap(cmap_branches)

figs_tracking = {}
for i, (gtr, probe, ev) in enumerate(zip(gtrs, probes, evals)):
    fig, ax = plt.subplots()
    fig.set_size_inches((10, 10))
    pitch = probe.pitch[0]
    evaluation = ev
    locations = gtr.locations
    template = gtr.template
    
    branch_colors = [cm(i / max_branches) for i in range(max_branches)]
    
    nplt.plot_neuron(morphology=str(morphology_path), plane='xy', alpha=0.1, ax=ax, position=cell_model.somapos,
                     exclude_sections=['axon'])
    nplt.plot_neuron(morphology=str(morphology_path), plane='xy', alpha=0.1, ax=ax, position=cell_model.somapos,
                     exclude_sections=['soma', 'apic', 'basal'], color='g')
    ax = gtr.plot_clean_branches(cmap=cmap_branches, plot_bp=False, plot_full_template=False,
                                 ax=ax, branch_colors=branch_colors)

    cm = plt.get_cmap(cmap_branches)

    ax.axis('equal')
    ax.axis('off')
    figs_tracking[str(pitch)] = fig

In [ ]:
if save_figures:
    for pitch, fig in figs_tracking.items():
        fig.savefig(fig_folder / f"{pitch}_tracking.png", dpi=600)